In [8]:
#import necessary packages
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import tree
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, GroupKFold

from sklearn.metrics import f1_score
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
import time

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
trlabels = pd.read_csv("train_labels.csv")
sample = pd.read_csv("sample_submission.csv")

In [4]:
trlabels['session'] = trlabels.session_id.apply(lambda s: int(s.split('_')[0]))
trlabels['question'] = trlabels.session_id.apply(lambda s: int(s.split('_')[1][1:]))

In [5]:
# Categorical features 
CATS = ['event_name', 'name','fqid', 'room_fqid', 'text_fqid']

# Numerical features
NUMS = ['elapsed_time','level','page','room_coor_x', 'room_coor_y', 
        'screen_coor_x', 'screen_coor_y', 'hover_duration']

# define the function to create the features
def feature_engineer(train):
    dfs = []
    for c in CATS:
        tmp = train.groupby(['session_id','level_group'])[c].agg('nunique')
        tmp.name = tmp.name + '_nunique'
        dfs.append(tmp)
    for c in NUMS:
        tmp = train.groupby(['session_id','level_group'])[c].agg('mean')
        dfs.append(tmp)
    for c in NUMS:
        tmp = train.groupby(['session_id','level_group'])[c].agg('std')
        tmp.name = tmp.name + '_std'
        dfs.append(tmp)
    df = pd.concat(dfs,axis=1)
    df = df.fillna(-1)
    df = df.reset_index()
    df = df.set_index('session_id')
    return df

In [6]:
new_train = feature_engineer(train)
FEATURES = [c for c in new_train.columns if c != 'level_group'] # all features except the target
print('We will train with', len(FEATURES) ,'features')
ALL_USERS = new_train.index.unique() # treat each index as a user
print('We will train with', len(ALL_USERS) ,'users info')

We will train with 21 features
We will train with 23562 users info


In [55]:

prediction = pd.DataFrame(data=np.zeros((len(ALL_USERS),2)), index=ALL_USERS)

print(prediction)

                     0    1
session_id                 
20090312431273200  0.0  0.0
20090312433251036  0.0  0.0
20090312455206810  0.0  0.0
20090313091715820  0.0  0.0
20090313571836404  0.0  0.0
...                ...  ...
22100215342220508  0.0  0.0
22100215460321130  0.0  0.0
22100217104993650  0.0  0.0
22100219442786200  0.0  0.0
22100221145014656  0.0  0.0

[23562 rows x 2 columns]


In [ ]:

# models = [LinearRegression(), ]
clf = LinearRegression()
#clf = 

gkf = GroupKFold(n_splits=10)
oof = pd.DataFrame(data=np.zeros((len(ALL_USERS),18)), index=ALL_USERS)
#prediction = pd.DataFrame(data=np.zeros((len(ALL_USERS),2)), index=ALL_USERS)

prediction = pd.DataFrame(data=np.zeros((len(ALL_USERS),18)), index=ALL_USERS)

# COMPUTE CV SCORE WITH 5 GROUP K FOLD
for i, (train_index, test_index) in enumerate(gkf.split(X=new_train, groups=new_train.index)):
    print('#'*25)
    print('### Fold',i+1)
    print('#'*25)
    
    # ITERATE THRU QUESTIONS 1 THRU 18
    for t in range(1,19):
        print(t,', ',end='')
        
        # USE THIS TRAIN DATA WITH THESE QUESTIONS
        if t<=3: grp = '0-4'
        elif t<=13: grp = '5-12'
        elif t<=22: grp = '13-22'
            
        # TRAIN DATA
        train_x = new_train.iloc[train_index]
        train_x = train_x.loc[train_x.level_group == grp]
        #print(train_x)
        train_users = train_x.index.values
        train_y = trlabels.loc[trlabels.question==t].set_index('session').loc[train_users]
        
#         print(train_y)
        
        # VALID DATA
        valid_x = new_train.iloc[test_index]
        valid_x = valid_x.loc[valid_x.level_group == grp]
        valid_users = valid_x.index.values
        valid_y = trlabels.loc[trlabels.question ==t].set_index('session').loc[valid_users]
        
#         print(valid_y)
#         print()
        # TRAIN MODEL
#         print("valid")
#         print(valid_y)
#         print()
#         print()
#         print()
        clf.fit(train_x[FEATURES].astype('float32'), train_y['correct'])
        
#         # SAVE MODEL, PREDICT VALID OOF
#         models[f'{grp}_{t}'] = clf
        
#         print(f'{grp}_{t}')
     
        oof.loc[valid_users, t-1] = clf.predict(valid_x[FEATURES].astype('float32'))
        prediction.loc[valid_users, t-1] = valid_y['correct']
        #print("predict")
        #print(valid_x[FEATURES].astype('float32'))
        #p = valid_y[['session_id', 'correct']]
        #prediction = pd.concat([p, prediction])
#         prediction = clf.predict(valid_x[FEATURES].astype('float32'))
        
# #         print(valid_y.shape)
#         print(valid_y)
# #         print(prediction.shape)
#         print(classification_report(valid_y, prediction))
    print()
print(oof)

In [ ]:
prediction

In [28]:
clf = tree.DecisionTreeClassifier(max_depth = 5)

gkf = GroupKFold(n_splits=10)
oof = pd.DataFrame(data=np.zeros((len(ALL_USERS),18)), index=ALL_USERS)
#prediction = pd.DataFrame(data=np.zeros((len(ALL_USERS),2)), index=ALL_USERS)

# prediction = pd.DataFrame(data=np.zeros((len(ALL_USERS),18)), index=ALL_USERS)

# COMPUTE CV SCORE WITH 5 GROUP K FOLD
for i, (train_index, test_index) in enumerate(gkf.split(X=new_train, groups=new_train.index)):
    print('#'*25)
    print('### Fold',i+1)
    print('#'*25)
    
    # ITERATE THRU QUESTIONS 1 THRU 18
    for t in range(1,19):
        print(t,', ',end='')
        
        # USE THIS TRAIN DATA WITH THESE QUESTIONS
        if t<=3: grp = '0-4'
        elif t<=13: grp = '5-12'
        elif t<=22: grp = '13-22'
            
        # TRAIN DATA
        train_x = new_train.iloc[train_index]
        train_x = train_x.loc[train_x.level_group == grp]
        #print(train_x)
        train_users = train_x.index.values
        train_y = trlabels.loc[trlabels.question==t].set_index('session').loc[train_users]
        
#         print(train_y)
        
        # VALID DATA
        valid_x = new_train.iloc[test_index]
        valid_x = valid_x.loc[valid_x.level_group == grp]
        valid_users = valid_x.index.values
        valid_y = trlabels.loc[trlabels.question ==t].set_index('session').loc[valid_users]
        
        clf.fit(train_x[FEATURES].astype('float32'), train_y['correct'])
        
        oof.loc[valid_users, t] = clf.predict(valid_x[FEATURES].astype('float32'))
        #prediction.loc[valid_users, t-1] = valid_y['correct']

    print()

#########################
### Fold 1
#########################
1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 , 
#########################
### Fold 2
#########################
1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 , 
#########################
### Fold 3
#########################
1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 , 
#########################
### Fold 4
#########################
1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 , 
#########################
### Fold 5
#########################
1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 , 
#########################
### Fold 6
#########################
1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 16 , 17 , 18 , 
#########################
### Fold 7
#########################
1 , 2 , 3 , 4 , 5 , 6 , 7 , 8 , 9 , 10 , 11 , 12 , 13 , 14 , 15 , 1

In [29]:
oof = oof.stack().reset_index()

In [30]:
oof.rename(columns = {'session_id':'session', 'level_1':'question',
                              0 :'correct'}, inplace = True)

In [31]:
oof['correct'] = oof['correct'].astype('int')

In [32]:
oof = oof.sort_values(by=['question'])

In [33]:
oof

,session,question,correct
0,20090312431273200,0,0
141246,21020115094940690,0,0
141265,21020115094968436,0,0
141284,21020115105814480,0,0
408253,22040413024042544,0,0
...,...,...,...
283574,21070420124156640,18,1
283593,21070500291927656,18,1
283612,21070507434025624,18,1
283460,21070417510735372,18,1


In [23]:
trlabels

,session_id,correct,session,question
0,20090312431273200_q1,1,20090312431273200,1
1,20090312433251036_q1,0,20090312433251036,1
2,20090312455206810_q1,1,20090312455206810,1
3,20090313091715820_q1,0,20090313091715820,1
4,20090313571836404_q1,1,20090313571836404,1
...,...,...,...,...
424111,22100215342220508_q18,1,22100215342220508,18
424112,22100215460321130_q18,1,22100215460321130,18
424113,22100217104993650_q18,1,22100217104993650,18
424114,22100219442786200_q18,1,22100219442786200,18


In [34]:
combined = pd.merge(trlabels[['session', 'correct', 'question']], oof, left_on='session', right_on='session', how='left')

In [35]:
combined.sort_values(by=['question_x', 'question_y'])

,session,correct_x,question_x,question_y,correct_y
0,20090312431273200,1,1,0,0
19,20090312433251036,0,1,0,0
38,20090312455206810,1,1,0,0
57,20090313091715820,0,1,0,0
76,20090313571836404,1,1,0,0
...,...,...,...,...,...
8058127,22100215342220508,1,18,18,1
8058146,22100215460321130,1,18,18,1
8058165,22100217104993650,1,18,18,1
8058184,22100219442786200,1,18,18,1


In [36]:
print(classification_report(combined['correct_x'], combined['correct_y'], digits = 3))

              precision    recall  f1-score   support

           0      0.334     0.222     0.267   2372055
           1      0.715     0.816     0.762   5686149

    accuracy                          0.641   8058204
   macro avg      0.525     0.519     0.514   8058204
weighted avg      0.603     0.641     0.616   8058204

